<span STYLE="font-size:150%"> 
    Z-Stack loader
</span>

Docker image: gnasello/slicer-env:2023-01-25 \
Latest update: 3 Feb 2023

Objective of the Notebook

- load .czi z-stack images in Slicer 

# Import libraries

In [1]:
from pathlib import Path
import slicer
from aicsimageio import AICSImage
from pandas import DataFrame

Resource 'XMLSchema.xsd' is already loaded


# User inputs

In [16]:
# File path of the z-stack file
zstack_file = 'GN_CHIR_d2_whole_2023-01-18.czi'

# 3D Slicer takes individual channels as volume nodes
channel = 'T PMT-T2' # brightfield

color = 'blue'

# Get the AICSImage object

In [3]:
img = AICSImage(zstack_file)  # selects the first scene found

## Print pixel sizes

In [4]:
x_res = img.physical_pixel_sizes.X  # returns the X dimension pixel size as found in the metadata
y_res = img.physical_pixel_sizes.Y  # returns the X dimension pixel size as found in the metadata
z_res = img.physical_pixel_sizes.Z  # returns the Z dimension pixel size as found in the metadata
size = [x_res, y_res, z_res]

In [5]:
# The lines below used to work but now they raise the error 
# "cannot get a schema for XML data, provide a schema argument"
# Check The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  
# To silence this warning, please provide the `parser` argument, specifying either 'lxml' 
# (to opt into the new behavior), or'xmlschema' (to retain the old behavior).

# metadata_dict = img.ome_metadata.dict()['images'][0]['pixels']
# x_unit = metadata_dict['physical_size_x_unit'].value
# y_unit = metadata_dict['physical_size_y_unit'].value
# z_unit = metadata_dict['physical_size_z_unit'].value
# unit = [x_unit, y_unit, z_unit]

unit = ['µm', 'µm', 'µm']

In [6]:
data = {
  "pixel_size": size,
  "unit": unit
}

rownames = ['x', 'y', 'z']
pixel_df = DataFrame(data, index = rownames)

pixel_df_mm = pixel_df.copy()
pixel_df_mm.pixel_size = pixel_df.pixel_size/1000
pixel_df_mm.unit = ['mm', 'mm', 'mm'] 

In [7]:
print('\n--- Image Dimensions')
print(img.dims)  # returns a Dimensions object

print('\n--- Image Channel Names')
print(img.channel_names)  # returns a list of string channel names found in the metadata

print('\n--- Image Pixel Physical Size Table')
print(pixel_df_mm)


--- Image Dimensions
<Dimensions [T: 1, C: 5, Z: 89, Y: 5213, X: 985]>

--- Image Channel Names
['Ch1-T1', 'ChS2-T1', 'ChS1-T2', 'Ch2-T2', 'T PMT-T2']

--- Image Pixel Physical Size Table
   pixel_size unit
x    0.001384   mm
y    0.001384   mm
z    0.001200   mm


## Get image data as numpy array

In [8]:
imgdata = img.get_image_data("CZYX", T=0)  # returns 4D CZYX numpy array

In [9]:
# imgdata.shape

In [10]:
channel_data = imgdata[img.channel_names.index(channel),:]

# Load image as Volume Node

Create a master volume node with geometry based on the input images

Instantiate and add a VolumeNode to the scene.
To create a volume from a numpy array, you need to initialize a ```vtkMRMLScalarVolumeNode``` [link](https://discourse.slicer.org/t/creating-volume-from-numpy/658/4)

In [11]:
masterVolumeNode = slicer.vtkMRMLScalarVolumeNode()

In [12]:
masterVolumeNode.SetSpacing(pixel_df_mm.pixel_size.to_list())

Importing images in czi file extension [link](https://discourse.slicer.org/t/importing-images-in-czi-file-extension/12291/1)

In [13]:
slicer.util.updateVolumeFromArray(masterVolumeNode, channel_data)

In [14]:
masterVolumeNode = slicer.mrmlScene.AddNode(masterVolumeNode)

In [15]:
slicer.util.setSliceViewerLayers(background=masterVolumeNode, fit=True)
masterVolumeNode.CreateDefaultDisplayNodes()

In [18]:
clrs = ['yellow','red','green','blue']

if color in clrs:
    lookup_table = 'vtkMRMLColorTableNode' + color.capitalize()
    displayNode = masterVolumeNode.GetDisplayNode()
    displayNode.SetAndObserveColorNodeID(lookup_table)

In [20]:
color='grey'

lookup_table = 'vtkMRMLColorTableNode' + color.capitalize()
displayNode = masterVolumeNode.GetDisplayNode()
displayNode.SetAndObserveColorNodeID(lookup_table)